# Aula05.Ex01 - Exemplo - Cubo - Transformação Geométrica 3D

### Primeiro, vamos importar as bibliotecas necessárias.

In [64]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
import time

### Inicializando janela

In [65]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(700, 700, "Programa", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)

### Shaders

Note que agora usamos vec3, já que estamos em 3D.

In [66]:
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

In [67]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [68]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [69]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [70]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [71]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [72]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [73]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Classe do Objeto

In [74]:
#matriz identidade
identity = np.array([1.0,  0.0, 0.0,     0, 
                    0.0,    1.0,   0.0, 0, 
                    0.0,    0.0,   1.0, 0, 
                    0.0,    0.0,   0.0, 1.0], np.float32)

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

#soma o valor das 2 coordenadas
#coord1: list[3]
#coord2: list[3]
def sum_coord(coord1, coord2):
    return [coord1[0]+coord2[0], coord1[1]+coord2[1], coord1[2]+coord2[2]]




class Object:
    #vertices = coordenadas dos vertices
    #cor = lista de cores
    #indice_cor = lista de tupla (vertice de inicio de um cor, quantos vertices esse cor tem)
    #transform = matriz de transformacao
    #coord = coordenadas do centro
    def __init__(self,vertices = [], color = [[0,0,0,0]], indice_color = [[0,0]], transform = identity, coordinate = [0,0,0]):
        self.vertices = vertices
        self.color = color
        self.indice_color = indice_color
        self.transform = transform
        self.coord = coordinate

    
    #retorna a coordenada do centro do objeto e a matriz transladada
    #retorna o matriz de transicao 
    #translate_coord: float[3]
    def translate(self, translate_coord):
        mat_translation = np.array([1.0,  0.0, 0.0, translate_coord[0], 
                                    0.0,    1.0,   0.0, translate_coord[1], 
                                    0.0,    0.0,   1.0, translate_coord[2], 
                                    0.0,    0.0,   0.0, 1.0], np.float32)
        self.coord = sum_coord(self.coord,translate_coord)
        self.transform = multiplica_matriz(mat_translation, self.transform)

    #retorna matriz rotacionada
    #rot_coord: float[3]
    def rotate(self, rot_coord):
        #calculo do cos/sen
        cos_x = math.cos(rot_coord[0])
        sin_x = math.sin(rot_coord[0])
        cos_y = math.cos(rot_coord[1])
        sin_y = math.sin(rot_coord[1])
        cos_z = math.cos(rot_coord[2])
        sin_z = math.sin(rot_coord[2])

        #matrizes
        mat_rotation_z = np.array([cos_z, -sin_z, 0.0, 0.0, 
                                    sin_z,  cos_z, 0.0, 0.0, 
                                    0.0,      0.0, 1.0, 0.0, 
                                    0.0,      0.0, 0.0, 1.0], np.float32)
        mat_rotation_x = np.array([1.0,   0.0,    0.0, 0.0, 
                                    0.0, cos_x, -sin_x, 0.0, 
                                    0.0, sin_x,  cos_x, 0.0, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
            
        mat_rotation_y = np.array([cos_y,  0.0, sin_y, 0.0, 
                                    0.0,    1.0,   0.0, 0.0, 
                                    -sin_y, 0.0, cos_y, 0.0, 
                                    0.0,    0.0,   0.0, 1.0], np.float32)
        
        #multiplicacao de matrizes
        minus_center = [-1 * x for x in self.coord]
        _,temp = self.translate(minus_center)
        self.transform = multiplica_matriz(temp,self.transform)
        self.transform = multiplica_matriz(mat_rotation_z,self.transform)
        self.transform = multiplica_matriz(mat_rotation_x,self.transform)
        self.transform = multiplica_matriz(mat_rotation_y,self.transform)
        _,temp = self.translate(self.coord)
        self.transform = multiplica_matriz(temp,self.transform)


    #retorna o matriz escalada
    #scale_coord: float[3]
    def scale(self,scale_coord):
        #matrizes
        mat_scale = np.array([scale_coord[0], 0.0, 0.0, 0.0, 
                                0.0,  scale_coord[1], 0.0, 0.0, 
                                0.0,      0.0, scale_coord[2], 0.0, 
                                0.0,      0.0, 0.0, 1.0], np.float32)
        
        #multiplicacao de matrizes
        minus_center = [-1 * x for x in self.coord]
        _,temp = self.translate(minus_center)
        self.trasform = multiplica_matriz(temp,self.trasform)
        self.trasform = multiplica_matriz(mat_scale,self.trasform)
        _,temp = self.translate(self.coord)
        self.trasform = multiplica_matriz(temp,self.trasform)

    #renderiza o objeto
    def render(self):
        #envia o matriz de transformacao para shader
        loc_transformation = glGetUniformLocation(program, "mat_transformation")
        glUniformMatrix4fv(loc_transformation, 1, GL_TRUE, self.transform)
        for i in range(0,len(self.color)):
            #cor
            glUniform4f(loc_color, self.color[i][0], self.color[i][1], self.color[i][2], self.color[i][3])
            #desenha   
            glDrawArrays(GL_TRIANGLE_STRIP, self.indice_color[i][0], self.indice_color[i][1])

### Preparando dados para enviar a GPU

Até aqui, compilamos nossos Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


In [75]:
#variaveis
total_vertices = 0 #numero total de vertices
vertices_list = [] #lista de coordenadas de vertices da forma (x,y,z)

#chao
chao_altura = -0.3
chao_marrom_vertice = [(-1,-1,-1),(1,-1,-1),(-1,chao_altura,-1),(1,chao_altura,-1)]
chao_marrom = Object(chao_marrom_vertice,
                     [[0.22,0.10,0,1]],
                     [[total_vertices,total_vertices+len(chao_marrom_vertice)]]
                     )
total_vertices += len(chao_marrom_vertice)
vertices_list.extend(chao_marrom_vertice)

chao_verde_vertice = [(-1,1,-1),(1,1,-1),(-1,chao_altura,-1),(1,chao_altura,-1)]
chao_verde = Object(chao_verde_vertice,
                     [[0.16,0.33,0,1]],
                     [[total_vertices,total_vertices+len(chao_verde_vertice)]]
                     )
total_vertices += len(chao_verde_vertice)
vertices_list.extend(chao_verde_vertice)


#transformacao para np.array
vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
vertices['position'] = np.array(vertices_list)


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot (buffer).

In [76]:
# Request a buffer slot from GPU
buffer_VBO = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [77]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)

### Associando variáveis do programa GLSL (Vertex Shader) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [78]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [79]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui <b> três </b> coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [80]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

### Vamos pegar a localização da variável color para que possamos definir a cor em nosso laço da janela!

In [81]:
loc_color = glGetUniformLocation(program, "color")

### Capturando eventos de teclado e modificando variáveis para a matriz de transformação

In [82]:
#codigo dos keys
KEY_P = 80

#variaveis
isMalha = False

def key_event(window,key,scancode,action,mods):
    global isMalha
    
    if key == KEY_P and action == glfw.PRESS: isMalha = not isMalha #malha
           
glfw.set_key_callback(window,key_event)

### Nesse momento, nós exibimos a janela!


In [83]:
glfw.show_window(window)

### Loop principal da janela.

In [84]:
glEnable(GL_DEPTH_TEST) ### importante para 3D

#variaveis


#transformacoes iniciais



while not glfw.window_should_close(window):

    #reset canvas
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)    
    glClearColor(1.0, 1.0, 1.0, 1.0)

    #representacao em malha
    if isMalha:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    else: 
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)

    #chao
    chao_marrom.render()
    chao_verde.render()
    
    glfw.swap_buffers(window)
    glfw.poll_events()

glfw.terminate()